In [1]:
from kadlu.geospatial.ocean import Ocean
from kadlu.geospatial.data_sources.data_util import reshape_2D
from kadlu.geospatial.data_sources.chs import Chs
from datetime import datetime
import numpy as np

In [2]:
kwargs = dict(
        start=datetime(2015, 1, 9), end=datetime(2015, 1, 9, 12),
        south=44,                   west=-64.5, 
        north=46,                   east=-62.5, 
        top=0,                      bottom=5000
    )

In [3]:
load_bathymetry = Chs().load_bathymetry(**kwargs)
type(load_bathymetry)

numpy.ndarray

#### loading bathymetry data from array of real values

In [7]:
t1 = datetime.now()
o = Ocean(load_bathymetry=load_bathymetry, cache=True, **kwargs)
t2 = datetime.now()
print(f'interpolations computed in {(t2-t1).seconds} seconds')

/home/matt_s/kadlu/kadlu/geospatial/data_sources/data_util.py:205: UserWarning: query data has been averaged across the time dimension for 3D interpolation.
to avoid this behaviour, use keyword argument 'time' instead of start/end
  warnings.warn("query data has been averaged across the time dimension "
/home/matt_s/kadlu/kadlu/geospatial/data_sources/data_util.py:205: UserWarning: query data has been averaged across the time dimension for 3D interpolation.
to avoid this behaviour, use keyword argument 'time' instead of start/end
  warnings.warn("query data has been averaged across the time dimension "
/home/matt_s/kadlu/kadlu/geospatial/data_sources/data_util.py:205: UserWarning: query data has been averaged across the time dimension for 3D interpolation.
to avoid this behaviour, use keyword argument 'time' instead of start/end
  warnings.warn("query data has been averaged across the time dimension "
/home/matt_s/kadlu/kadlu/geospatial/data_sources/data_util.py:205: UserWarning: query

interpolations computed in 46 seconds


the same data can then be loaded from the cache quickly

In [8]:
t1 = datetime.now()
o = Ocean(load_bathymetry=load_bathymetry, cache=True, **kwargs)
t2 = datetime.now()
print(f'interpolations loaded from cache in {(t2-t1).seconds}.{int((t2-t1).microseconds/1000)} seconds')

interpolations loaded from cache in 0.335 seconds


#### loading bathymetry data using array of 'realistic' testing data

In [39]:
load_bathymetry = np.array((
    np.random.randint(0, 500, 100) *-1,
    np.array([lat for lat in range(4400, 4600, 2)]) / 100,
    np.array([lon for lon in range(-6450, -6250, 2)]) / 100
))
load_bathymetry.shape

(3, 100)

In [40]:
o = Ocean(load_bathymetry=load_bathymetry, cache=True, **kwargs)

/home/matt_s/kadlu/kadlu/geospatial/data_sources/data_util.py:205: UserWarning: query data has been averaged across the time dimension for 3D interpolation.
to avoid this behaviour, use keyword argument 'time' instead of start/end
  warnings.warn("query data has been averaged across the time dimension "
/home/matt_s/kadlu/kadlu/geospatial/data_sources/data_util.py:205: UserWarning: query data has been averaged across the time dimension for 3D interpolation.
to avoid this behaviour, use keyword argument 'time' instead of start/end
  warnings.warn("query data has been averaged across the time dimension "
/home/matt_s/kadlu/kadlu/geospatial/data_sources/data_util.py:205: UserWarning: query data has been averaged across the time dimension for 3D interpolation.
to avoid this behaviour, use keyword argument 'time' instead of start/end
  warnings.warn("query data has been averaged across the time dimension "
/home/matt_s/kadlu/kadlu/geospatial/data_sources/data_util.py:205: UserWarning: query

In [41]:
o.temp(lat=[45], lon=[-63], z=[100])

-5.99980926950176

#### reshaping bathymetry data is the largest bottleneck in the pipeline

In [42]:
t1 = datetime.now()
bathy_reshaped = reshape_2D(Chs().load_bathymetry, **kwargs)
t2 = datetime.now()
print(f'bathymetry data prepared in {(t2-t1).seconds} seconds')

bathymetry data prepared in 47 seconds
